In [1]:
import networkx as nx
import tqdm
import pickle as pkl
import json
import powerlaw

In [2]:
entries = {}
internal_papers = {}

#with open("merged_internal-citations.json") as json_file:
with open("citation_network.json") as json_file:
    entries = json.load(json_file)
internal_papers = entries['arxiv_papers']



In [3]:
del entries

In [4]:
print(f"#Internal : {len(internal_papers)} + #External: {len(entries['external_papers'])} = {len(internal_papers)+len(entries['external_papers'])}")



NameError: name 'entries' is not defined

In [5]:
print(internal_papers['1302.6135'][0])

['astro-ph/0609602', '1106.1746', 'astro-ph/0609501', 'astro-ph/0510237', '1003.5625', '1004.1069', '0805.2133', '0811.0492', 'astro-ph/0501552', 'astro-ph/0210157', '0807.0573', '1501.04226', 'astro-ph/0011497', 'astro-ph/0205417', '0806.3004', '1003.0618', '1011.5768', '0808.4015', '1009.5425', '1008.1727', '0901.1098', 'astro-ph/9812448', '1008.1761', '1101.1698', 'astro-ph/0507718', '1202.4523', 'astro-ph/9809387', 'astro-ph/0601049', 'astro-ph/0602087', 'astro-ph/0210694', '0907.2250', 'astro-ph/0504082', '0908.1239', '0911.5718', 'astro-ph/0503381', '1111.3761', 'astro-ph/0304350', '0807.3971', 'astro-ph/0312113', '0807.1515', '1201.6415', '1003.5876', 'astro-ph/0601518', '0708.1752', 'astro-ph/0101378', '1105.2577', '0803.1484', '1109.3447', '1302.6135', '1111.2487']


In [8]:
intit = iter(internal_papers.keys())
for i in range(10):
    print(next(intit))

1302.6135
1510.04023
0904.4671
0912.1308
1103.4102
1402.1367
1604.07202
1108.4230
0902.0235
1502.05819


In [6]:
G = nx.DiGraph()
nodelist = internal_papers.keys()
for node in tqdm.tqdm(nodelist):
    if(not (node in G)):
        G.add_node(node)
    for reference in internal_papers[node][0]:
        G.add_edge(node,reference)  
    for reference in internal_papers[node][1]:
        G.add_edge(node,reference)  

100%|██████████| 1793794/1793794 [01:36<00:00, 18569.91it/s]


In [9]:
del internal_papers

In [19]:
indata = []
outdata = []
for node in tqdm.tqdm(G.nodes()):
    indata.append(G.in_degree(node))
    outdata.append(G.out_degree(node))
#result = powerlaw.Fit(data)

100%|██████████| 7363967/7363967 [00:14<00:00, 519590.16it/s]


In [20]:
inresult = powerlaw.Fit(indata)
outresult = powerlaw.Fit(outdata)

Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit


In [21]:
print(inresult.power_law.alpha)
print(inresult.power_law.xmin)
print(outresult.power_law.alpha)
print(outresult.power_law.xmin)


2.708639248721764
204.0
3.969735224624406
75.0


In [13]:
node_count = len(G.nodes())
edge_count = len(G.edges())
isolate_count = len(list(nx.isolates(G)))
print(f"# Nodes: {node_count}")
print(f"# Edges: {edge_count}")
print(f"# Isolates: {isolate_count}")
print(f"# %Isolates: {isolate_count / node_count}")
print("")
# Avg. degree <k> calculation
print(f"Average degree k: {2* edge_count / node_count}")
# % WCC calculation
largest_cc = max(nx.weakly_connected_components(G), key=len)
print(f"# WCC: {len(largest_cc)}")
print(f"% WCC: {len(largest_cc) / node_count}")

# Highest in-and outdegree nodes for cap on co-cit / bib-coup
max_out_deg = max(G.out_degree, key=lambda e: e[1])
max_in_deg = max(G.in_degree, key=lambda e: e[1])
print(max_out_deg)
print(max_out_deg[1]**2)
print(max_in_deg)
print(max_in_deg[1]**2)


# Nodes: 7363967
# Edges: 48826943
# Isolates: 6583
# %Isolates: 0.0008939475149739264

Average degree k: 13.261043402285752
# WCC: 7298539
% WCC: 0.9911151149916886
('quant-ph/0012089', 6155)
37884025
('phys', 132236)
17486359696


In [ ]:
co_citations = set()
for node in tqdm.tqdm(G.nodes()):
    for ref in G.neighbors(node):
        for ref2 in G.neighbors(node):
            if ref < ref2:
                co_citations.add((ref, ref2))

In [ ]:
print(f"# Co-citations: {len(co_citations)}")
it = iter(co_citations)
for i in range(10):
    print(next(it))

In [ ]:
#pkl.dump(co_citations, open("co_citations.p", "wb"))
#co_citations = pkl.load(open("co_citations.p", "rb"))

In [11]:
sbib_coupling = set()
for node in tqdm.tqdm(G.nodes()):
    for p in G.predecessors(node):
        for p2 in G.predecessors(node):s
            if p < p2:
                bib_coupling.add((p, p2))


100%|██████████| 3346320/3346320 [07:23<00:00, 7541.87it/s]


In [6]:
#pkl.dump(bib_coupling, open("bib_coupling.p", "wb"))

In [12]:
print(f"# Bib-couples: {len(bib_coupling)}")
it = iter(bib_coupling)
for i in range(10):
    print(next(it))

# Bib-couples: 581068567
(('1912.02689v2', 2698118), ('2007.01650v1', 2986849))
(('0712.0155v1', 160655), ('1506.04128v2', 291635))
(('1103.2683v1', 1483416), ('1307.5932v1', 2071974))
(('1903.12519v2', 2237794), ('2009.12981v2', 3188085))
(('1910.14162v1', 2253678), ('2007.14454v1', 2794148))
(('1712.01038v1', 1949875), ('2009.06206v2', 2828881))
(('1202.5375v4', 1917636), ('hep-th/0403104v3', 2621140))
(('0905.0589v1', 2070352), ('hep-th/0208182v4', 3339189))
(('1607.01205v1', 1534037), ('1904.07516v1', 3287488))
(('1709.08112v1', 632143), ('1806.07438v3', 1813626))
